In [1]:
import cv2
import os
import csv
import random
import numpy as np
import pandas as pd
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt

In [4]:
# Reading images of characters without background. 

path = '/home/alena/jupyter/jupyter/Optical-Braille-Recognition-System/dataset/chars_wo_background'

n = 0
X = []

for root, dirs, files in os.walk(path):
    files = [file[:-4] for file in files]
    files = sorted(map(int, files))
    for filename in files:
        filename = str(filename) + '.jpg'
        example = cv2.resize(cv2.imread(path + '/' + filename, 0), (40, 64))
        X = np.append(X, example)
        
        n += 1
        
X = X.reshape(n, 64, 40, 1).astype(np.uint8)
print(X.shape)

(350, 64, 40, 1)


In [5]:
# Reading "noise" characters (decimal numbers).

path = '/home/alena/jupyter/jupyter/Optical-Braille-Recognition-System/dataset/noise'

ni = 0
noise = []

for root, dirs, files in os.walk(path):
    files = [file[:-4] for file in files]
    files = sorted(map(int, files))
    for filename in files:
        filename = str(filename) + '.jpg'
        example = cv2.resize(cv2.imread(path + '/' + filename, 0), (40, 64))
        noise = np.append(noise, example)
        ni += 1
        
noise = noise.reshape(ni, 64, 40, 1).astype(np.uint8)

In [3]:
labels = pd.read_excel('/home/alena/jupyter/jupyter/Optical-Braille-Recognition-System/dataset/braille_characters_labels_full.xlsx', header=None)
labels = np.array(labels)

In [6]:
# Width and height of each sheet.
w, h = 1500, 2000

# Width and height of each symbol.
H_SYM = 64-4
W_SYM = 40-4

#sheet[0:48,0:30] = example
#plt.imshow(sheet, cmap='gray')
#plt.show()
#cv2.imwrite('krya.png', sheet)

# Each page contains 5 to 20 lines.
N_EXAMPLES = 201
MIN_LINES = 5
MAX_LINES = 20

# Distances between side characters and page edges.
H_PAGE_INDENT = 70
W_PAGE_INDENT = 80

# Distance between characters inside the word.
SYM_INDENT = 10

# Distances between words vertically and horizontally. 
H_WORD_INDENT = 30
W_WORD_INDENT = 50

# Each line contains 3 to 6 words or 0 words with probability of 5%.
# Each word consists of:
# 1-2 letters - 40%, 
# 3-7 letters - 50%,
# 8-11 letters - 10%.

# Each symbol has random offset in the range of 0 to 3 vertically and horizontally.

In [7]:
sheets_labels = np.array([])

for i in range(N_EXAMPLES):
    
    sheet = np.full((h, w, 1), 255)
    x_pointer = W_PAGE_INDENT
    y_pointer = H_PAGE_INDENT
    
    n_lines = random.randint(MIN_LINES, MAX_LINES)
    
    for j in range(n_lines):
        
        if random.random() < 0.05:
            continue
        
        n_words = random.randint(3, 6)
        
        for k in range(n_words):
            
            r = random.random()
            if r < 0.1:
                n_letters = random.randint(8, 11)
            elif r < 0.4:
                n_letters = random.randint(1, 2)
            else:
                n_letters = random.randint(3, 7)
            
            # If the word is too long to fit in line, skip it.
            if x_pointer < w - n_letters * (W_SYM + SYM_INDENT) - W_PAGE_INDENT - 2:
                for q in range(n_letters):
                
                    x_offset = random.randint(-2, 2)
                    y_offset = random.randint(-2, 2)
                
                    x = x_pointer + x_offset
                    y = y_pointer + y_offset
                    
                    if random.random() < 0.02:
                        char = noise[random.randint(0, 8)]
                    else:
                        index = random.randint(0, 349)
                            
                        char = X[index]
                    
                        if i == 0 and q == 0:
                            sheets_labels = np.array([i, x, y, x + W_SYM, y + H_SYM, labels[index]])
                        else:
                            sheets_labels = np.vstack((sheets_labels, np.array([i, x, y, x + W_SYM, y + H_SYM, labels[index]])))
                        
                        dots = char[char < 210].flatten()
                        avg = np.sum(dots) / len(dots)
                        if avg > 150:
                            char[char < 210] = char[char < 210] - 30
                        elif avg < 140:
                            char[char < 210] = char[char < 210] + 30
                            
                    #print(y, y+H_SYM, x, x+W_SYM, char.shape)
                    
                    # Applying minor rotation.
                    p = random.randint(0, 5)
                    aug = iaa.Affine(rotate=(-p, p))
                    char = aug(image=char)
                    char = char[2:H_SYM+2, 2:W_SYM+2]
                    
                    sheet[y:y+H_SYM, x:x+W_SYM] = char
                    x_pointer += (W_SYM + SYM_INDENT)
                
                x_pointer += W_WORD_INDENT
                
        x_pointer = W_PAGE_INDENT
        y_pointer += (H_SYM + H_WORD_INDENT)     
        
    flat = sheet.flatten()    
    white = np.where(np.logical_and(flat <= 255, flat >= 210)) 
    np.put(flat, white, 170)
    sheet = flat.reshape((2000, 1500, 1)).astype(np.uint8)
    
    # Applying some gaussian noise.
    p = random.uniform(0.05, 0.1)
    aug = iaa.AdditiveGaussianNoise(scale=(0, p*255))
    sheet = aug(image=sheet)
    
    cv2.imwrite('/home/alena/jupyter/jupyter/Optical-Braille-Recognition-System/dataset/dot/' + str(i) + '.png', sheet)


<ipython-input-7-115f5271c682>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sheets_labels = np.array([i, x, y, x + W_SYM, y + H_SYM, labels[index]])
<ipython-input-7-115f5271c682>:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sheets_labels = np.vstack((sheets_labels, np.array([i, x, y, x + W_SYM, y + H_SYM, labels[index]])))


In [8]:
# Writing labels.

with open('/home/alena/jupyter/jupyter/Optical-Braille-Recognition-System/dataset/dots_labels_1.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(sheets_labels)